In [2]:
from tempfile import tempdir
import SimpleITK as sitk
import numpy as np
import numpy.ma as ma
from matplotlib import pyplot
import matplotlib.pyplot as plt
import os
import sys
#import seaborn as sns
import UsefulFunctions as UF
import pandas as pd
import radiomics

In [4]:
image = "D:\\data\\prostateMR_radiomics\\nifti\\20fractions\\0000257\\MR36\\BaseImages\\0000257_MR36_Raw.nii"
mask = "D:\\data\\prostateMR_radiomics\\nifti\\20fractions\\0000257\\MR36\\Masks\\0000257_MR36_shrunk_pros.nii"

def MaskedMeanMed(image_url, mask_url):
    '''
    Returns mean and std of masked image
    Input - image path and mask path
    Output - mean and median (floats)
    '''

    image = sitk.ReadImage(image_url)
    image_array = sitk.GetArrayFromImage(image)
    
    mask = sitk.ReadImage(mask_url)
    mask_array = sitk.GetArrayFromImage(mask) 
    mask_array = mask_array / (np.max(mask_array))
    if image_array.shape != mask_array.shape:
        mean = np.nan
        med = np.nan
    
    else:
        masked_image = ma.array(image_array, mask=np.logical_not(mask_array), keep_mask=False, hard_mask=True)
        mean = ma.mean(masked_image)
        masked_image = masked_image[~masked_image.mask]
        med = ma.median(masked_image)
       
    return mean, med

mean, med = MaskedMeanMed(image, mask)
print(mean, med)

80.84616825932171 77.0


In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
df = pd.DataFrame(np.random.randint(0,100,size=(10, 3)), columns=["Pat", "Mean", "Median"])
df = df.melt(id_vars=["Pat"], value_vars=["Mean", "Median"])
df.rename(columns={"value": "Signal"})


,Pat,variable,Signal
0,60,Mean,21
1,99,Mean,53
2,18,Mean,46
3,55,Mean,77
4,66,Mean,74
5,94,Mean,49
6,46,Mean,90
7,57,Mean,17
8,63,Mean,30
9,68,Mean,19


In [76]:
csv_url = "D:\\data\\Aaron\\ProstateMRL\\Data\\MRLPacks\\Features_v2\\All_pyRad_Features.csv"
all_df = pd.read_csv(csv_url)
all_df = all_df.loc[(all_df.Normalisation == "Raw") & (all_df.Region == "Prostate")]
SABR = all_df.loc[all_df.Treatment == "SABR"]
#SABR = SABR.loc[SABR.Normalisation == "Raw"]
print(SABR.DaysDiff.value_counts())

f20 = all_df.loc[all_df.Treatment == "20fractions"]
print("--------------------------")
print(f20.DaysDiff.value_counts())

0     10
7      8
4      8
2      8
9      3
11     2
5      2
10     1
8      1
Name: DaysDiff, dtype: int64
--------------------------
0     10
4      5
21     4
11     4
13     3
6      3
25     3
22     3
8      2
30     2
15     2
18     2
20     2
27     2
28     2
14     2
29     1
7      1
2      1
1      1
Name: DaysDiff, dtype: int64


In [43]:
g = sns.lineplot(df,linestyle="variable")

ValueError: If using all scalar values, you must pass an index

In [5]:
from radiomics import featureextractor

root = UF.DataRoot()

parameters = root + "Aaron\\ProstateMRL\\Data\\MRLPacks\\ExtractionParams\\Run1.yaml"
extractor = featureextractor.RadiomicsFeatureExtractor(parameters)


In [30]:
temp_results = pd.Series(extractor.execute(image, mask))#, label=mask_value))
temp_df = pd.DataFrame()

temp_df = temp_df.append(temp_results, ignore_index=True)
temp_df["PatID"] = "Test"
temp_df["Scan"] = "hello"
#temp_df.loc[;, ~temp_df.columns.str.contains("diagnostic")]
#print(temp_df.head())
temp2 = pd.DataFrame()
temp2 = temp2.append(temp_results, ignore_index=True)
temp2["PatID"] = "test2"
temp2["Scan"] = "goodbye"
temp_df = temp_df.append(temp2)
#trans = temp_results.transpose()

print(temp_df)


  diagnostics_Configuration_EnabledImageTypes  \
0                            {'Original': {}}   
0                            {'Original': {}}   

                  diagnostics_Configuration_Settings  \
0  {'minimumROIDimensions': 2, 'minimumROISize': ...   
0  {'minimumROIDimensions': 2, 'minimumROISize': ...   

  diagnostics_Image-original_Dimensionality  \
0                                        3D   
0                                        3D   

            diagnostics_Image-original_Hash  \
0  00784881ee652a511075a3a79bfc67b75fa52f72   
0  00784881ee652a511075a3a79bfc67b75fa52f72   

   diagnostics_Image-original_Maximum  diagnostics_Image-original_Mean  \
0                               573.0                        44.463728   
0                               573.0                        44.463728   

   diagnostics_Image-original_Minimum diagnostics_Image-original_Size  \
0                                 0.0                 (480, 480, 300)   
0                             

In [15]:
trans = temp_results.transpose()

print(trans)#.shape)

diagnostics_Versions_PyRadiomics                                                              v3.0.1
diagnostics_Versions_Numpy                                                                    1.19.5
diagnostics_Versions_SimpleITK                                                                 2.0.0
diagnostics_Versions_PyWavelet                                                                 1.1.1
diagnostics_Versions_Python                                                                    3.6.5
diagnostics_Configuration_Settings                 {'minimumROIDimensions': 2, 'minimumROISize': ...
diagnostics_Configuration_EnabledImageTypes                                         {'Original': {}}
diagnostics_Image-original_Hash                             00784881ee652a511075a3a79bfc67b75fa52f72
diagnostics_Image-original_Dimensionality                                                         3D
diagnostics_Image-original_Spacing                 (0.8333396911621094, 0.8333396911621094,

In [56]:
scaninfo = pd.read_csv("D:\\\\data\\Aaron\\ProstateMRL\\Data\\PatientKey.csv")
scaninfo.head()
patIDs = scaninfo.Patient.unique()
scaninfo.reset_index(drop=True)
#print(patIDs)
s, f = 0,0
res_df = pd.DataFrame()
treatments = scaninfo.Treatment.unique()
for t in treatments:
    scans = scaninfo.loc[scaninfo["Treatment"] == t] 
    treatmentIDs = scans.Patient.unique()
    print(len(treatmentIDs))
    for i in treatmentIDs:
        pat_df = scans.loc[scans["Patient"] == i]
        #print(pat_df.head())
        dates = pat_df.DateofScan.unique()
        sorted_df = pd.DataFrame()
        #print(i, dates)
        for j in dates:
            date_df = pat_df.loc[pat_df["DateofScan"] == j]
            #print(date_df.head(1))
            sorted_df = sorted_df.append(date_df.head(1))
        
        sorted_df["Scan"] = sorted_df["Scan"].str.replace(" MR", "")
        sorted_df["Scan"] = pd.to_numeric(sorted_df["Scan"])
        sorted_df = sorted_df.sort_values(["Scan"])
        sorted_df["Scan"] = "MR" + sorted_df["Scan"].astype(str)
        res_df = res_df.append(sorted_df)

print(res_df)
    

res_df.reset_index(drop=True, inplace=True)
res_df.to_csv("D:\\data\\Aaron\\ProstateMRL\\Data\\PatientKey_sorted.csv")

10
10
     Patient    Treatment  Scan  DateofScan
11       653         SABR   MR6    20111207
0        653         SABR  MR14    20111211
3        653         SABR  MR17    20111214
6        653         SABR  MR21    20111216
8        653         SABR  MR23    20111218
..       ...          ...   ...         ...
127   312108  20fractions  MR35    20191007
129   312108  20fractions  MR37    20191008
132   312108  20fractions  MR50    20191014
134   312108  20fractions  MR52    20191015
135   312108  20fractions  MR62    20191018

[98 rows x 4 columns]


In [69]:
mean_std = pd.read_csv("D:\data\Aaron\ProstateMRL\Data\MRLPacks\InterFractionChanges_v2\All_std.csv")

regions = mean_std.Region.unique()
norms = mean_std.Normalisation.unique()
patients = mean_std.PatID.unique()
print(patients)

for r in regions:
    region_df = mean_std.loc[mean_std.Region == r]

    norm_df = region_df.loc[region_df.Normalisation == "Raw"]
    stds = []
    means = []
    perc90 = []
    for p in patients:
        pat_df = norm_df.loc[norm_df.PatID == p]
        #print(r, p, np.mean(pat_df["StdSignal"]))
        std = np.mean(pat_df["StdSignal"])
        stds.append(std)
        mean = np.mean(pat_df["MeanSignal"])
        means.append(mean)
        percs = np.mean(pat_df["90thPerc"])
        perc90.append(percs)
    
    print(r, np.mean(stds), np.mean(means))#, np.mean(perc90))


            



[   213    231    257    292    626    695   1464 106958 251842 312108
    653    713    752    826   1088   1089   1118   1303   1307]
Prostate 32.44661970076236 61.848311894988704
Glute 24.06402131614344 12.809070375169222
Psoas 10.061523848040691 14.5320377283742
